# Vérification de l'intégrité de la clé LDA et transformation de la base validations
Ce NoteBook a pour but de vérifier l'intégriter de la clé LDA, seule code disponible pour faire une jointure avec les autres données d'Île De France Mobilités (_IDFM_).

LDA est la clé la plus exploitable de la base, cependant, d'après nos recherches dans la documentation d'_IDFM_ elle peut être associée à ZdC, c'est à dire aux _Zone de Correspondance_. Cela peut donc englober plusieurs stations (particulièrement sur les pôles de correspondances) et il peut également y avoir des stations très grandes qui sont découpées en différents pôles de correspondance (comme Chatelet/Halles par exemple).

Il s'agit donc de mettre au clair ici ces premiers constats et dans un second temps, de réaliser les aggrégations correctes pour avoir des données exploitables et qui ont du sens.

## Packages et importation des données

In [23]:
import pandas as pd
from _importation_donnees import donnees_validation_23
from _pre_proces_Insee_IDFM import data_INSEE_stations

In [24]:
data=donnees_validation_23()

In [25]:
stations_RER=data_INSEE_stations()

## Analyse de la clé LDA

In [26]:
print(data['LIBELLE_ARRET'].nunique())
print(data['lda'].nunique())

747
726


On remarque que dans la base brute il y a une différence de 21 entre le nombre d'arrêt et le nombre de LDA. Cela peut provenir de deux sources potentielles :
1. Un même nom de station est associé à des LDA différent : cela peut être le cas dans les gros pôle multimodal ;
2. Un même LDA peut être associé à des stations différentes :
   - On peut imaginer qu'une même station soit écrite de plusieures manières différentes (il s'agit juste d'une mauvaise gestion de la base),
   - Il peut également s'agir d'erreur de numérotation,
   - C'est un gros pôle de correspondance qui engloble plusieurs stations.
  
Nous faisons ces vérifications sur la base brute fournis par _IDFM_ mais notre attention se portera plus particulièrement sur les stations de _RER_ qui sont au coeur de notre étude.

### Même station associée à des LDA différents

In [27]:
#On se restreint aux colonnes lda et LIBELLE_ARRET
sub_data = data[['lda', 'LIBELLE_ARRET']]
#Suppression des doublons
unique_stations = sub_data.drop_duplicates()
#On aggrège et on filre pour avoir les stations qui ont structement plus de 1 LDA
multi_lda_stations = unique_stations.groupby('LIBELLE_ARRET').filter(lambda x: x['lda'].nunique() > 1)
#On trie par ordre alphabétique pour faciliter la lecture
multi_lda_stations.sort_values(by='LIBELLE_ARRET')

,lda,LIBELLE_ARRET
931,71607,BERCY
6471,73652,BERCY
327,71264,CHATELET
1671,73794,CHATELET
871,74040,MONTPARNASSE
2611,71139,MONTPARNASSE


Il y a donc très peu de stations dans ce cas et seule Chatelet est concerné par les _RER_. On ne s'interesse donc qu'à la station de Chatelet. Nous observons maintenant comment sont structurés les données des stations.

In [28]:
stations_RER[stations_RER['id_ref_ZdC']==71264]

,Geo Point,Geo Shape,gares_id,nom_long,nom_so_gar,nom_su_gar,id_ref_ZdC,nom_ZdC,id_ref_ZdA,nom_ZdA,...,ind_25_39,ind_40_54,ind_55_64,ind_65_79,ind_80p,ind_inc,pre_IDF,select,expt,


In [29]:
stations_RER[stations_RER['id_ref_ZdC']==73794]

,Geo Point,Geo Shape,gares_id,nom_long,nom_so_gar,nom_su_gar,id_ref_ZdC,nom_ZdC,id_ref_ZdA,nom_ZdA,...,ind_25_39,ind_40_54,ind_55_64,ind_65_79,ind_80p,ind_inc,pre_IDF,select,expt,


In [30]:
stations_RER[stations_RER['nom_long']=="Châtelet-Les Halles"]

,Geo Point,Geo Shape,gares_id,nom_long,nom_so_gar,nom_su_gar,id_ref_ZdC,nom_ZdC,id_ref_ZdA,nom_ZdA,...,ind_25_39,ind_40_54,ind_55_64,ind_65_79,ind_80p,ind_inc,pre_IDF,select,expt,
39,"48.861822271863645, 2.3470126872387564","{""coordinates"": [2.347012687238756, 48.8618222...",163,Châtelet-Les Halles,NaN,NaN,474151,Châtelet les Halles,45102,Châtelet les Halles,...,8501.5,6004.0,2991.0,2856.0,863.0,20.0,1.0,1.0,1.0,NaN
40,"48.861822271863645, 2.3470126872387564","{""coordinates"": [2.347012687238756, 48.8618222...",164,Châtelet-Les Halles,NaN,NaN,474151,Châtelet les Halles,45102,Châtelet les Halles,...,8501.5,6004.0,2991.0,2856.0,863.0,20.0,1.0,1.0,1.0,NaN
41,"48.861822271863645, 2.3470126872387564","{""coordinates"": [2.347012687238756, 48.8618222...",170,Châtelet-Les Halles,NaN,NaN,474151,Châtelet les Halles,45102,Châtelet les Halles,...,8501.5,6004.0,2991.0,2856.0,863.0,20.0,1.0,1.0,1.0,NaN


In [31]:
stations_RER[stations_RER['nom_long']=="Châtelet-Les Halles"].loc[:,['res_com']]

,res_com
39,RER A
40,RER B
41,RER D


La station de RER correspond à "Châtelet-Les Halles" et non "Châtelet" (qui ne donne que la fréquentation des métros). Cependant, dans la base des données de validation cette disctinction n'existe pas.

In [32]:
data[data['lda']==474151]

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,lda,CATEGORIE_TITRE,NB_VALD


Dans le reste de la base les stations en de _RER_ en correspondance avec le métro n'ont pas de LDA différent : seule le pôle de _Châtelet - Les Halles_ est assez important pour être divisé en différentes Zones de Correspondance (*ZdC*§*LDA*) qui nous servirons pour faire la jointure avec les autres données *IDFM*. 

Afin d'avoir une analyse cohérante, nous décidons de regrouper toutes les données de validation du pôle sous une seule station (ce qui est la cas dans les fait grâce aux couloirs de correspondance). De plus, nous associons le numéro qui permettra de faire la jointure avec la base des stations (i.e. 474151).

Pour commencer on remplace le *LDA* des stations Châtelet et des Halles par le nouveau (il nous servira ensuite à faire l'aggrégation par *LDA*.

In [33]:
data.loc[data['LIBELLE_ARRET'].isin(['CHATELET', 'LES HALLES']), 'lda'] = 474151

In [34]:
data[data['lda']==474151].head()

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,lda,CATEGORIE_TITRE,NB_VALD
327,2023-06-27,100,110,165,CHATELET,474151,FGT,629
472,2023-06-27,100,110,460,LES HALLES,474151,?,931
473,2023-06-27,100,110,460,LES HALLES,474151,AUTRE TITRE,1648
474,2023-06-27,100,110,460,LES HALLES,474151,NAVIGO,24278
650,2023-05-12,100,110,165,CHATELET,474151,NAVIGO JOUR,167


In [35]:
data[data['lda']==71264].head()

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,lda,CATEGORIE_TITRE,NB_VALD


In [36]:
data[data['lda']==73794].head()

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,lda,CATEGORIE_TITRE,NB_VALD


In [37]:
#On se restreint aux colonnes lda et LIBELLE_ARRET
sub_data = data[['lda', 'LIBELLE_ARRET']]
#Suppression des doublons
unique_stations = sub_data.drop_duplicates()
#On aggrège et on filre pour avoir les stations qui ont structement plus de 1 LDA
multi_lda_stations = unique_stations.groupby('LIBELLE_ARRET').filter(lambda x: x['lda'].nunique() > 1)
#On trie par ordre alphabétique pour faciliter la lecture
multi_lda_stations.sort_values(by='LIBELLE_ARRET')

,lda,LIBELLE_ARRET
931,71607,BERCY
6471,73652,BERCY
871,74040,MONTPARNASSE
2611,71139,MONTPARNASSE


La manipulation a été faite correctement : il n'y a plus de ligne avec les anciens codes que nous avons remplacé. Il ne reste plus que Bercy et Montparnasse, mais elles ne sont pas des stations de *RER*.

Il nous faut maintenant voir s'il y a des mêmes LDA associés à des stations différentes et décider de la meilleure méthode d'aggrégation.

### Même LDA associé à des stations différentes

In [38]:
#On se restreint aux colonnes lda et LIBELLE_ARRET
sub_data = data[['lda', 'LIBELLE_ARRET']]
#Suppression des doublons
unique_lda = sub_data.drop_duplicates()
#On aggrège et on filre pour avoir les stations qui ont structement plus de 1 LDA
multi_stations_lda = unique_lda.groupby('lda').filter(lambda x: x['LIBELLE_ARRET'].nunique() > 1)
#On trie par ordre alphabétique pour faciliter la lecture
multi_stations_lda.shape

(48, 2)

Il y a donc 48 lignes sur lesquelles nous devons décider ce qu'il faut faire.

In [39]:
multi_stations_lda.sort_values(by='lda')

,lda,LIBELLE_ARRET
233,63244,MASSY-PALAIS.
2456,63244,MASSY PALAISEAU
1211,63320,MASSY VERRIERES
235,63320,MASSY-VERRIER.
93,64382,ST NOM LA BRETE
1141,64382,Saint-Nom-la-Bretèche Forêt de Marly T13
1139,64589,Saint-Germain-en-Laye T13
182,64589,ST-GERMAIN
111,64622,ST GERMAIN GCO
1143,64622,Lisière Pereire T13


1. On identifie clairement des stations pour lesquelles il s'agit simplement d'une mauvaise appellation de certaines stations : *Saint-Cyr T13* et *ST CYR* correspondent bien à la même *Zone de Correspondance* (*ZdC*), tout comme *SAINT-MICHEL* et *SAINT-MICHEL NOTRE DAME*. il n'y a aucun problème puisque la jointure se fera avec le *LDA* qui correspond bien à une seule *Zone de Correspondance*.
   
2. On remarque quelques numérotations surprenantes au niveau du *T13* puisque certaines stations ont le même *LDA* alors qu'elles sont très espacées. On ne considère pas ces stations car elles ne correspondent pas à des arrets de *RER* et ne seront pas gardées lors de la jointure. De même les arrêts Brochant et Pont Cardinet ont le même *LDA* alors qu'ils ne sont pas à proximité immédiate. Ces deux stations sont en dehors du réseau de *RER* donc cela ne pose pas de problème particulier.

3. Le seul cas qui n'est pas extrèmement rigoureux pour notre étude est celui de la station Javel et Mirabeau qui ne sont pas en correspondance directe et qui partage pourtant le même *LDA*. On décide simplement de supprimer les lignes avec la stations Mirabeau pour éviter de gonfler artificiellement les validations au niveau de la station *Javel* (*RER C*).

In [40]:
data = data[data['LIBELLE_ARRET'] != 'MIRABEAU']

## Modification de la base

Maintenant que la base est correctement modifiée, il ne reste plus qu'à aggréger la base des données de validation 2023 avec *LDA* pour chaque jour et catégorie de titre. L'information de la station sera donc entièrement contenue dans le *LDA* et nous pourrons alors faire la jointure avec les données des stations d'*IDFM*.

In [41]:
aggregated_data = data.groupby(['JOUR', 'lda', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

On vérifie maintenant que l'aggrégation s'est faite correctement en utilisant la station de *La Défense*.

Au niveau désagrégé on obteint :

In [42]:
data.sort_values(by=['lda', 'CATEGORIE_TITRE', 'JOUR'], ascending=False).loc[data['lda']==71517].head()

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,lda,CATEGORIE_TITRE,NB_VALD
1015211,2023-06-30,100,110,414,LA DEFENSE,71517,TST,608
1016857,2023-06-30,800,854,394,DEFENSE,71517,TST,2537
1017057,2023-06-30,810,801,393,LA DEFENSE,71517,TST,3539
666813,2023-06-29,810,801,393,LA DEFENSE,71517,TST,4033
817922,2023-06-29,100,110,414,LA DEFENSE,71517,TST,621


Après l'aggrégation on obtient :

In [43]:
aggregated_data.sort_values(by=['lda', 'CATEGORIE_TITRE', 'JOUR'], ascending=False).loc[aggregated_data['lda']==71517].head()

,JOUR,lda,CATEGORIE_TITRE,NB_VALD
1037351,2023-06-30,71517,TST,6684
1031480,2023-06-29,71517,TST,6867
1025600,2023-06-28,71517,TST,5796
1019742,2023-06-27,71517,TST,6550
1013881,2023-06-26,71517,TST,7553


Et, pour le 30 juin 2023 on a bien que 608+2537+3539=6684 !

In [44]:
aggregated_data.shape

(1039315, 4)